In [13]:
# import necessary configurations
import configparser

config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

# amazon aws
KEY = config.get('AWS', 'key')
SECRET = config.get('AWS', 'secret')

# Redshift
DWH_DB = config.get('DWH', 'DWH_DB')
DWH_DB_USER = config.get('DWH', 'DWH_DB_USER')
DWH_DB_PASSWORD = config.get('DWH', 'DWH_DB_PASSWORD')
DWH_PORT = config.get('DWH', 'DWH_PORT')
DWH_CLUSTER_TYPE = config.get('DWH', 'DWH_CLUSTER_TYPE')
DWH_NUM_NODES = config.get('DWH', 'DWH_NUM_NODES')
DWH_NODE_TYPE = config.get('DWH', 'DWH_NODE_TYPE')
DWH_IAM_ROLE_NAME = config.get('DWH', 'DWH_IAM_ROLE_NAME')
DWH_CLUSTER_IDENTIFIER = config.get('DWH', 'DWH_CLUSTER_IDENTIFIER')
DWH_SCHEMA = config.get('DWH', 'DWH_SCHEMA')
DWH_LOG_STAGING_TABLE = config.get('DWH', 'DWH_LOG_STAGING_TABLE')
DWH_SONG_STAGING_TABLE = config.get('DWH', 'DWH_SONG_STAGING_TABLE')
DWH_REGION = config.get('DWH', 'DWH_REGION')

# s3
LOG_JSON_FORMAT = config.get('S3', 'LOG_JSON_FORMAT')
S3_BUCKET_LOG_JSON_PATH = config.get('S3', 'S3_BUCKET_LOG_JSON_PATH')
S3_BUCKET_SONG_JSON_PATH = config.get('S3', 'S3_BUCKET_SONG_JSON_PATH')

In [ ]:
%load_ext sql

In [ ]:
# save your DWH_ENDPOINT here:
DWH_ROLE_ARN = '<fill_it_up_here>'
DWH_ENDPOINT = '<fill_it_up_here>'

In [ ]:
# creating connection to your DB in Amazon Redshift
conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT, DWH_DB)
print(conn_string)
%sql $conn_string

#### Check the schema before starting. There should be total 7 tables (including 2 staging, 1 fact and 5 dimensions)

In [ ]:
%%sql
select t.table_name
from information_schema.tables t
where t.table_schema = 'public'
order by t.table_name;

#### Schema Diagram

![title](img/schema.png)

#### Most popular songs heard so far!

In [ ]:
qry = """
      SELECT s.title, count(*) as count
      FROM {}.songplays sp
      INNER JOIN {}.songs s ON s.song_id = sp.song_id
      GROUP BY s.title
      ORDER BY count DESC, s.title ASC;
      """.format(DWH_SCHEMA, DWH_SCHEMA)

%sql $qry

#### Most popular artists and their songs heard so far!

In [ ]:
qry = """
      SELECT ar.name, s.title, count(*) as count
      FROM {}.songplays sp
      INNER JOIN {}.songs s ON s.song_id = sp.song_id
      INNER JOIN {}.artists ar ON ar.artist_id = sp.artist_id
      GROUP BY ar.name, s.title
      ORDER BY count DESC, ar.name, s.title ASC;
      """.format(DWH_SCHEMA, DWH_SCHEMA, DWH_SCHEMA)

%sql $qry